# Task 2: Sentiment and Thematic Analysis
**Quantify review sentiment and identify themes to uncover satisfaction drivers and pain points**

---

## 📋 Table of Contents
1. [Setup & Installation](#setup)
2. [Load Your Clean Data](#load-data)
3. [Sentiment Analysis](#sentiment)
4. [Bank-Wise Analysis](#bank-analysis)
5. [Thematic Analysis](#thematic)
6. [Visualizations](#visualizations)
7. [Export Results](#export)
8. [KPI Validation](#kpi)

---

## 1. Setup & Installation <a name="setup"></a>

### Install Required Packages
Run this cell only once

In [3]:
# Install required packages (run once)
!pip install pandas numpy matplotlib seaborn nltk textblob scikit-learn -q
!pip install spacy -q
!python -m spacy download en_core_web_sm -q

print("✅ Packages installed successfully!")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
✅ Packages installed successfully!


In [4]:
import nltk
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt')

print("✅ NLTK data downloaded successfully!")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ermias/nltk_data...


✅ NLTK data downloaded successfully!


[nltk_data] Downloading package stopwords to /home/ermias/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ermias/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Import Libraries

In [22]:
# Data manipulation
import pandas as pd
import numpy as np
from collections import Counter

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# NLP
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from textblob import TextBlob

# Advanced NLP (optional)
try:
    import spacy
    SPACY_AVAILABLE = True
except ImportError:
    SPACY_AVAILABLE = False
    print("⚠️ spaCy not available. Some features may be limited.")

# Feature extraction
from sklearn.feature_extraction.text import TfidfVectorizer

# System
import os
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 8)

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [23]:
from src.core.configLoader import config_loader


In [24]:
path_processed = config_loader.load_config('scraping')['base_processing']['paths']['processed_reviews']
path_processed

'data/processed/reviews_processed.csv'

In [25]:
df = pd.read_csv(path_processed)
df

,review_id,review_text,rating,review_date,review_year,review_month,bank_code,bank_name,user_name,thumbs_up,text_length,source
0,3463230e-f9f7-4be3-a632-fdd8d017ce84,🙏👍,5,2025-11-29,2025,11,BOA,Bank of Abyssinia,Yasin Alemu,0,2,Google Play
1,a6cbfa34-f2b1-4a16-96b6-c94f58cea76f,Very Good,5,2025-11-28,2025,11,BOA,Bank of Abyssinia,Wariyo Dida,0,9,Google Play
2,fc67d12c-92e2-45aa-a9e0-011f58a583bc,goof,5,2025-11-28,2025,11,BOA,Bank of Abyssinia,Hailegebrail Tegegn,0,4,Google Play
3,11306fb9-5571-4950-8d32-604c5402242f,good!,5,2025-11-28,2025,11,BOA,Bank of Abyssinia,Tsegay ab,0,5,Google Play
4,809c46d2-730e-446a-9061-2a45e978ad9d,good jop,5,2025-11-27,2025,11,BOA,Bank of Abyssinia,Yohanis Fikadu,0,8,Google Play
...,...,...,...,...,...,...,...,...,...,...,...,...
1195,b3c8405c-96a7-4b5e-884c-76c97c530c34,good,5,2025-05-09,2025,5,Dashen,Dashen Bank,fenta abebayehu,1,4,Google Play
1196,e1c1214a-8bc1-45db-bc49-3d51dddc6b88,Amazing app super easy to use and best design....,5,2025-05-09,2025,5,Dashen,Dashen Bank,Yitbarek Gossaye,1,56,Google Play
1197,8d1d472b-2bae-4749-b089-5632108ade02,its the best ever,5,2025-05-09,2025,5,Dashen,Dashen Bank,natnael abera,1,17,Google Play
1198,749851ed-72d7-4f1c-8e5e-27dd822b5008,nice,5,2025-05-08,2025,5,Dashen,Dashen Bank,Mehammed Amin,1,4,Google Play


In [21]:
df.columns

Index(['review_id', 'review_text', 'rating', 'review_date', 'review_year',
       'review_month', 'bank_code', 'bank_name', 'user_name', 'thumbs_up',
       'text_length', 'source'],
      dtype='object')

In [26]:
sia = SentimentIntensityAnalyzer()

In [27]:
def get_sentiment(text):
    score = sia.polarity_scores(str(text))['compound']
    if score >= 0.05:
        return 'positive', score
    elif score <= -0.05:
        return 'negative', score
    else:
        return 'neutral', score

In [29]:
df['sentiment_label'], df['sentiment_score'] = zip(*df['review_text'].apply(get_sentiment))

# 4. Save results
df.to_csv('reviews_with_sentiment.csv', index=False)
print("✅ Saved: reviews_with_sentiment.csv")
a
# 5. Show bank totals
print("\n📊 BANK SENTIMENT TOTALS:")
for bank in df['bank_name'].unique():
    bank_data = df[df['bank_name'] == bank]
    total = len(bank_data)
    pos = (bank_data['sentiment_label'] == 'positive').sum()
    print(f"{bank}: {total} reviews, {pos} positive ({pos/total*100:.0f}%)")

✅ Saved: reviews_with_sentiment.csv

📊 BANK SENTIMENT TOTALS:
Bank of Abyssinia: 400 reviews, 187 positive (47%)
Commercial Bank of Ethiopia: 400 reviews, 230 positive (57%)
Dashen Bank: 400 reviews, 256 positive (64%)


In [30]:
# 2. AGGREGATE BY RATING  
print("\nBY RATING:")
print("----------")
for rating in sorted(df['rating'].unique()):
    rating_data = df[df['rating'] == rating]
    avg_sentiment = rating_data['sentiment_score'].mean()
    count = len(rating_data)
    print(f"{rating}-star: {count} reviews, avg sentiment = {avg_sentiment:.2f}")



BY RATING:
----------
1-star: 263 reviews, avg sentiment = -0.15
2-star: 46 reviews, avg sentiment = 0.05
3-star: 67 reviews, avg sentiment = 0.17
4-star: 88 reviews, avg sentiment = 0.33
5-star: 736 reviews, avg sentiment = 0.38


In [32]:
# ============================================
# COMPLETE THEMATIC ANALYSIS WORKFLOW
# ============================================

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
from collections import defaultdict
import re

# -------------------------------------------------
# 1. LOAD AND PREPARE DATA
# -------------------------------------------------
def load_and_preprocess_data(filepath='bank_reviews.csv'):
    """
    Load review data and preprocess text
    """
    # Load data
    reviews = df
    
    # Basic preprocessing
    reviews['cleaned_text'] = reviews['review_text'].fillna('').astype(str)
    
    # Remove special characters but keep important punctuation for n-grams
    reviews['cleaned_text'] = reviews['cleaned_text'].apply(
        lambda x: re.sub(r'[^\w\s\-\.@#]', ' ', x.lower())
    )
    
    # Remove extra whitespace
    reviews['cleaned_text'] = reviews['cleaned_text'].apply(
        lambda x: ' '.join(x.split())
    )
    
    return reviews

# -------------------------------------------------
# 2. KEYWORD EXTRACTION WITH TF-IDF
# -------------------------------------------------
def extract_keywords_tfidf(reviews, text_column='cleaned_text', max_features=150):
    """
    Extract significant keywords using TF-IDF
    """
    # Initialize TF-IDF Vectorizer with banking-specific stop words
    banking_stop_words = ['bank', 'account', 'money', 'would', 'like', 'get', 'one', 'use']
    
    tfidf = TfidfVectorizer(
        max_features=max_features,
        stop_words='english',
        ngram_range=(1, 3),          # Extract unigrams, bigrams, and trigrams
        min_df=2,                    # Minimum document frequency
        max_df=0.8,                  # Maximum document frequency (avoid too common)
        analyzer='word',
        token_pattern=r'\b[a-zA-Z]{3,}\b',  # Words with at least 3 characters
        vocabulary=None
    )
    
    # Fit and transform
    tfidf_matrix = tfidf.fit_transform(reviews[text_column])
    keywords = tfidf.get_feature_names_out()
    tfidf_scores = np.array(tfidf_matrix.sum(axis=0)).flatten()
    
    # Create keyword DataFrame with scores
    keyword_df = pd.DataFrame({
        'keyword': keywords,
        'tfidf_score': tfidf_scores,
        'frequency': np.array(tfidf_matrix.astype(bool).sum(axis=0)).flatten()
    }).sort_values('tfidf_score', ascending=False)
    
    # Get top keywords by score
    top_keywords = keyword_df.head(50)
    
    return tfidf, tfidf_matrix, keyword_df, top_keywords

# -------------------------------------------------
# 3. ENHANCE WITH spaCy NAMED ENTITY RECOGNITION
# -------------------------------------------------
def extract_entities_spacy(reviews, text_column='cleaned_text'):
    """
    Extract banking-specific entities using spaCy
    """
    # Load spaCy model
    try:
        nlp = spacy.load("en_core_web_sm")
    except:
        # If model not installed, install it first
        import subprocess
        subprocess.run(["python", "-m", "spacy", "download", "en_core_web_sm"])
        nlp = spacy.load("en_core_web_sm")
    
    entities = defaultdict(list)
    
    # Process each review
    for text in reviews[text_column].head(100):  # Process first 100 for efficiency
        doc = nlp(text)
        
        for ent in doc.ents:
            if ent.label_ in ['ORG', 'PRODUCT', 'MONEY', 'CARDINAL']:
                entities[ent.label_].append(ent.text.lower())
    
    # Count entity frequencies
    entity_counts = {}
    for label, texts in entities.items():
        entity_counts[label] = pd.Series(texts).value_counts().head(10)
    
    return entity_counts

# -------------------------------------------------
# 4. MANUAL/RULE-BASED THEME CLUSTERING
# -------------------------------------------------
def cluster_keywords_into_themes(keyword_df, top_n=50):
    """
    Manually cluster keywords into 3-5 banking themes
    """
    # Get top keywords for clustering
    top_keywords = keyword_df.nlargest(top_n, 'tfidf_score')['keyword'].tolist()
    
    # Define theme categories with pattern matching
    theme_patterns = {
        'ACCOUNT_ACCESS_SECURITY': [
            r'login', r'password', r'access', r'security', r'authenticat', 
            r'locked', r'reset', r'biometric', r'face id', r'fingerprint',
            r'two factor', r'verification', r'security code', r'cant login'
        ],
        'TRANSACTION_PERFORMANCE': [
            r'transfer', r'transaction', r'payment', r'slow', r'fast',
            r'instant', r'pending', r'failed', r'processed', r'wire',
            r'direct deposit', r'mobile deposit', r'check deposit',
            r'money transfer', r'payment processing'
        ],
        'USER_INTERFACE_EXPERIENCE': [
            r'app', r'ui', r'ux', r'interface', r'design',
            r'easy to use', r'intuitive', r'navigation', r'crash',
            r'bug', r'glitch', r'loading', r'slow app', r'update',
            r'mobile banking', r'dashboard', r'layout'
        ],
        'CUSTOMER_SUPPORT': [
            r'customer service', r'support', r'help', r'call',
            r'wait', r'chat', r'representative', r'agent',
            r'response', r'complaint', r'issue resolved',
            r'phone support', r'live chat', r'escalation'
        ],
        'FEES_FINANCIAL_FEATURES': [
            r'fee', r'charge', r'cost', r'overdraft',
            r'interest', r'rate', r'apr', r'reward',
            r'cashback', r'credit card', r'loan',
            r'savings', r'checking', r'monthly fee',
            r'atm fee', r'foreign transaction'
        ]
    }
    
    # Cluster keywords into themes
    theme_keywords = defaultdict(list)
    theme_scores = defaultdict(float)
    
    for keyword in top_keywords:
        keyword_lower = keyword.lower()
        assigned = False
        
        for theme, patterns in theme_patterns.items():
            for pattern in patterns:
                if re.search(pattern, keyword_lower, re.IGNORECASE):
                    theme_keywords[theme].append(keyword)
                    
                    # Get TF-IDF score for this keyword
                    kw_score = keyword_df[keyword_df['keyword'] == keyword]['tfidf_score'].values
                    if len(kw_score) > 0:
                        theme_scores[theme] += kw_score[0]
                    
                    assigned = True
                    break
            
            if assigned:
                break
        
        # If no theme matched, add to "OTHER"
        if not assigned:
            theme_keywords['OTHER'].append(keyword)
    
    return theme_keywords, theme_scores, theme_patterns

# -------------------------------------------------
# 5. TOPIC MODELING (OPTIONAL ENHANCEMENT)
# -------------------------------------------------
def apply_topic_modeling(reviews, text_column='cleaned_text', n_topics=5):
    """
    Apply LDA topic modeling for theme validation
    """
    from sklearn.decomposition import LatentDirichletAllocation
    from sklearn.feature_extraction.text import CountVectorizer
    
    # Create document-term matrix
    vectorizer = CountVectorizer(
        max_features=1000,
        stop_words='english',
        ngram_range=(1, 2),
        min_df=2
    )
    
    dtm = vectorizer.fit_transform(reviews[text_column])
    feature_names = vectorizer.get_feature_names_out()
    
    # Apply LDA
    lda = LatentDirichletAllocation(
        n_components=n_topics,
        random_state=42,
        max_iter=10,
        learning_method='online'
    )
    
    lda_topics = lda.fit_transform(dtm)
    
    # Get top words for each topic
    topic_keywords = []
    for topic_idx, topic in enumerate(lda.components_):
        top_features_ind = topic.argsort()[-10:][::-1]
        top_features = [feature_names[i] for i in top_features_ind]
        topic_keywords.append(top_features)
    
    return lda, topic_keywords, lda_topics

# -------------------------------------------------
# 6. MAIN EXECUTION PIPELINE
# -------------------------------------------------
def main():
    """
    Complete thematic analysis pipeline
    """
    print("=" * 60)
    print("BANK REVIEW THEMATIC ANALYSIS")
    print("=" * 60)
    
    # Step 1: Load data
    print("\n1. Loading and preprocessing data...")
    reviews = load_and_preprocess_data('bank_reviews.csv')
    print(f"   Loaded {len(reviews)} reviews")
    
    # Step 2: Extract keywords with TF-IDF
    print("\n2. Extracting keywords using TF-IDF...")
    tfidf, tfidf_matrix, keyword_df, top_keywords = extract_keywords_tfidf(reviews)
    
    print("\n   Top 20 Keywords by TF-IDF Score:")
    print("   " + "-" * 40)
    for idx, row in top_keywords.head(20).iterrows():
        print(f"   {idx+1:2d}. {row['keyword']:30s} (Score: {row['tfidf_score']:.4f})")
    
    # Step 3: Extract entities with spaCy
    print("\n3. Extracting banking entities with spaCy...")
    entity_counts = extract_entities_spacy(reviews)
    
    print("\n   Top Entities Found:")
    print("   " + "-" * 40)
    for label, counts in entity_counts.items():
        print(f"   {label}: {', '.join(counts.head(5).index.tolist())}")
    
    # Step 4: Cluster into themes
    print("\n4. Clustering keywords into themes...")
    theme_keywords, theme_scores, theme_patterns = cluster_keywords_into_themes(keyword_df)
    
    print("\n   Themes Identified (with keyword counts):")
    print("   " + "-" * 40)
    for theme, keywords in theme_keywords.items():
        if keywords:  # Only show themes with keywords
            print(f"   {theme.replace('_', ' ').title():25s}: {len(keywords):2d} keywords")
            if len(keywords) > 0:
                print(f"     Sample: {', '.join(keywords[:5])}")
    
    # Step 5: Optional topic modeling
    print("\n5. Applying topic modeling (LDA) for validation...")
    try:
        lda, topic_keywords, lda_topics = apply_topic_modeling(reviews, n_topics=5)
        
        print("\n   LDA Topics (Top Keywords):")
        print("   " + "-" * 40)
        for i, keywords in enumerate(topic_keywords):
            print(f"   Topic {i+1}: {', '.join(keywords[:5])}")
    except Exception as e:
        print(f"   Topic modeling skipped: {e}")
    
    # Step 6: Generate actionable insights
    print("\n" + "=" * 60)
    print("ACTIONABLE INSIGHTS & RECOMMENDATIONS")
    print("=" * 60)
    
    # Calculate theme prevalence
    total_score = sum(theme_scores.values())
    
    for theme, score in sorted(theme_scores.items(), key=lambda x: x[1], reverse=True):
        if score > 0:
            prevalence = (score / total_score) * 100
            keywords = theme_keywords.get(theme, [])
            
            print(f"\n{theme.replace('_', ' ').title()}:")
            print(f"  Prevalence: {prevalence:.1f}% of significant feedback")
            print(f"  Key Issues: {', '.join(keywords[:5])}")
            
            # Generate recommendations
            if 'ACCESS' in theme:
                print("  Recommendation: Improve login flow and add biometric options")
            elif 'TRANSACTION' in theme:
                print("  Recommendation: Optimize transfer speeds and clear pending statuses")
            elif 'INTERFACE' in theme:
                print("  Recommendation: Update app UI and fix crashing issues")
            elif 'SUPPORT' in theme:
                print("  Recommendation: Reduce wait times and train support staff")
            elif 'FEES' in theme:
                print("  Recommendation: Review fee structure and offer fee-free options")
    
    # Return analysis results
    results = {
        'reviews': reviews,
        'keyword_df': keyword_df,
        'theme_keywords': dict(theme_keywords),
        'theme_scores': dict(theme_scores),
        'entity_counts': entity_counts
    }
    
    return results

# -------------------------------------------------
# 7. EXPORT RESULTS
# -------------------------------------------------
def export_results(results, output_file='thematic_analysis_results.csv'):
    """
    Export analysis results to CSV
    """
    # Create summary DataFrame
    summary_data = []
    
    for theme, keywords in results['theme_keywords'].items():
        if keywords:
            for keyword in keywords:
                # Get keyword info
                kw_info = results['keyword_df'][results['keyword_df']['keyword'] == keyword]
                if not kw_info.empty:
                    tfidf_score = kw_info['tfidf_score'].values[0]
                    frequency = kw_info['frequency'].values[0]
                    
                    summary_data.append({
                        'theme': theme.replace('_', ' ').title(),
                        'keyword': keyword,
                        'tfidf_score': tfidf_score,
                        'frequency': frequency,
                        'prevalence_percentage': (tfidf_score / results['keyword_df']['tfidf_score'].sum()) * 100
                    })
    
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_csv(output_file, index=False)
    print(f"\nResults exported to: {output_file}")
    
    return summary_df

# -------------------------------------------------
# EXECUTE THE ANALYSIS
# -------------------------------------------------
if __name__ == "__main__":
    # Run the complete analysis
    analysis_results = main()
    
    # Export results
    export_results(analysis_results)
    
    print("\n" + "=" * 60)
    print("ANALYSIS COMPLETE")
    print("=" * 60)
    print("\nNext Steps:")
    print("1. Review the exported CSV file for detailed insights")
    print("2. Validate themes with business stakeholders")
    print("3. Prioritize issues based on prevalence and impact")
    print("4. Track improvements with sentiment analysis over time")

BANK REVIEW THEMATIC ANALYSIS

1. Loading and preprocessing data...
   Loaded 1200 reviews

2. Extracting keywords using TF-IDF...

   Top 20 Keywords by TF-IDF Score:
   ----------------------------------------
   59. good                           (Score: 156.3678)
    8. app                            (Score: 121.1389)
   20. best                           (Score: 58.6278)
   92. nice                           (Score: 46.8196)
   14. bank                           (Score: 40.6634)
   74. like                           (Score: 23.4649)
   150. wow                            (Score: 22.8846)
   52. excellent                      (Score: 22.6363)
   17. banking                        (Score: 22.2009)
   60. good app                       (Score: 20.5360)
   137. use                            (Score: 20.2597)
   10. application                    (Score: 19.0527)
   146. working                        (Score: 18.4348)
   61. great                          (Score: 18.3925)
   54. fast  